# Code Cooker
Simple data analysis notebook on google colab like Advanced Data Analysis(Code Interpreter).

Reference: [claude_manim.ipynb](https://colab.research.google.com/github/wmoto-ai/claude-manim-notebook/blob/main/claude_manim.ipynb#scrollTo=mF8X8Q3LyZc9)

Install packages

In [ ]:
%cd /root/notebooks

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('.config')

CLAUDE_API_KEY = config.get('claude_api_key', 'key')

Import libraries and setup api

In [ ]:
import anthropic
import random
import time
from IPython.core.magic import register_cell_magic

client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

Define function

In [ ]:
MAX_RETRAY_COUNT = 3

@register_cell_magic
def _(line, cell):
    system_prompt = f"""あなたはJupyterNotebookを使うPythonデータサイエンティストです。
    入力された指示をもとに、データ分析を実施してください。
    回答には、1つのPythonコードブロック（マジックコマンド、クラス定義、コンストラクタを含む）のみを含め```pythonで囲ってください。
    限界を超えてがんばってください。うまくいったらチップあげます。
    """

    user_prompt = cell
    filename = line

    if filename != "":
        system_prompt += f"""
        # 入力データ:ファイルパス
        {filename}
        なお、データはUTF-8で読み込んでください。
        """

    messages = [
        {"role": "user", "content": user_prompt}
    ]

    prompt_w_history.extend(messages)

    retry_count = 0
    while retry_count < MAX_RETRAY_COUNT:
        try:
            response = client.messages.create(
                max_tokens=2048,
                messages=prompt_w_history,
                model="claude-3-opus-20240229",
                system=system_prompt
            )
        except Exception as e:
            print(e)
            time.sleep(3)
            continue

        assistant_response = response.content[0].text

        print("APIからの応答:", assistant_response)

        python = ""
        if "```python" in assistant_response and "```" in assistant_response:
            python = assistant_response.split("```python")[1].split("```")[0]

        print("抽出されたPythonコード:", python)

        try:
            exec(python)
            prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            break
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            retry_count += 1
            if retry_count < MAX_RETRAY_COUNT:
                prompt_w_history.append({"role": "user", "content": f"エラーが発生しました。エラーメッセージ: {e}\nコードを修正して再度実行してください。"})

    print("コードの実行が完了しました。" if retry_count < MAX_RETRAY_COUNT else "コードの修正に失敗しました。再実行回数が上限に達しました。")

Initialize prompt

In [ ]:
prompt_w_history = []

Execute

In [ ]:
%%_
kaggleのtitanicを分析してください

In [ ]:
%%_ train.csv
kaggleのtitanicを分析してください。対象のデータはtrain.csvです。

In [ ]:
%%_
可視化してください。

In [ ]:
%%_
架空の国の地域ごとの人口比率のデータを可視化してください。日本語でお願いします。